In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import os
import pandas as pd
import numpy as np
import random
import torch
import transformers
import matplotlib.pyplot as plt
import seaborn as sns

load all predictions

In [7]:
# load all predictions
dir_path = "../local/preds/hybrid-best"
# key = category id, value = data frame
predictions = {}  

for filename in os.listdir(dir_path):
    category_id = filename[:-4]  # strip .csv
    filepath = os.path.join(dir_path, filename)

    # store (category id, predictions)
    predictions[category_id] = pd.read_csv(filepath)
    
predictions["grocery"]

,Category Name,Category ID,Name,Image Name,Pred,Confidence
0,Alcohol,alcohol,saint brendan's irish cream liqueur,46892593.jpeg,alcohol,0.999837
1,Household,household,festive voice waterglobe (1 ct),59249357.jpeg,household,0.769288
2,Vitamins,vitamins,aura cacia eucalyptus 100% pure essential oil ...,65693796.jpeg,vitamins,0.998741
3,Personal Care,personal_care,hollywood beauty coconut oil (2 oz),91619698.jpeg,personal_care,0.959489
4,Pantry,pantry,jell-o cherry gelatin dessert (6 oz),3535125.jpeg,pantry,0.955524
...,...,...,...,...,...,...
16273,Meat & Fish,meat_and_fish,bar s sliced bacon (12 oz),49092778.jpeg,meat_and_fish,0.998832
16274,Household,household,sharpie marker retractable fine point fine (2 ct),19984289.jpeg,household,0.999931
16275,Snacks,snacks,endangered species bar oat mlk dk chocolate rc...,81544706.jpeg,candy,0.801836
16276,Pantry,pantry,"maries, dressing, poppy seed",64042401.jpeg,pantry,0.999975


Visualize examples of an L2 class incorrectly predicted as another L2 class.

In [20]:
def visualize_errors(l1, correct_l2, incorrect_l2):
    preds = predictions[l1]
    mask = ((preds["Category ID"] == correct_l2) & (preds["Pred"] == incorrect_l2)) | ((preds["Category ID"] == incorrect_l2) & (preds["Pred"] == correct_l2))
    return preds[mask].sort_values(["Category ID", "Confidence"], ascending=False)[["Category ID", "Name", "Pred", "Confidence"]]

In [21]:
print(visualize_errors("snacks", "chips", "popcorn").to_string())

     Category ID                                                                                Name     Pred  Confidence
1279     popcorn                                      pop chips potato chip snack barbeque (6.25 oz)    chips    0.997761
434      popcorn      old outch tostados white corn restaurante style premium tortilla chips (14 oz)    chips    0.995561
257      popcorn  pirates booty rice and corn puffs baked aged white cheddar lunch packs - 12-0.5 oz    chips    0.554991
661      popcorn  pirates booty rice and corn puffs baked aged white cheddar lunch packs - 12-0.5 oz    chips    0.554991
204        chips                                                   better made cheese popcorn (9 oz)  popcorn    0.979122
1552       chips                             cracker jack caramel coated popcorn & peanuts (2.87 oz)  popcorn    0.972576
1479       chips                               safe + fair sweet & salty kettle popcorn quinoa chips  popcorn    0.963187
1347       chips        

In [24]:
print(visualize_errors("pantry", "condiments", "sauces").to_string())

     Category ID                                                        Name        Pred  Confidence
1938      sauces                         heinz tomato ketchup simply (20 oz)  condiments    0.998362
334       sauces                  sir kensingtons spicy brown mustard (9 oz)  condiments    0.990694
1647      sauces                   chosen foods classic organic mayo (12 oz)  condiments    0.970673
854       sauces                                  kraft tartar sauce (12 oz)  condiments    0.918698
1385      sauces                                  kraft tartar sauce (12 oz)  condiments    0.918698
2110      sauces  essential everyday sandwich sauce, creamy chipotle (10 oz)  condiments    0.806261
236       sauces                  bookbinders prepared horseradish (9.75 oz)  condiments    0.590590
273   condiments                                 franks red hot sauce (5 oz)      sauces    0.992196
209   condiments                             huy fong sriracha sauce (28 oz)      sauces   

In [25]:
print(visualize_errors("baby_and_child", "baby_food", "toddler_snacks").to_string())

        Category ID                                                                                 Name       Pred  Confidence
400  toddler_snacks                                          happy baby banana & pumpkin superfood puffs  baby_food    0.980166
247  toddler_snacks  stonyfield organic yobaby apple & blueberry whole milk yogurt 6 count (6 ct / 4 oz)  baby_food    0.837877
